<a href="https://colab.research.google.com/github/susumu2357/US-patent-analysis/blob/master/patent_search_trial_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Data storage authentication
Please execute the below cell, copy and paste the authentication code.

You can execute the cell by pressing Shift+Enter or clicking the play button at left of the cell.

In [0]:
from google.colab import auth
auth.authenticate_user()

#2. Load data
In this trial model, 6440 patent publications are used as candidate proior arts.

In [0]:
#@title Please execute this cell. { display-mode: "form" }

import pandas as pd
import numpy as np
import math
import gzip
import pickle
from gensim.models import Word2Vec
import tensorflow as tf
tf.enable_eager_execution()

from google.colab import widgets
import urllib.parse

filename = "app3000_grant6000.model"
!gsutil cp gs://mlstudy-phys/data/{filename} ./
w2v = Word2Vec.load("app3000_grant6000.model")

filename = "app3000_grant6000_W.pkl.gz"
!gsutil cp gs://mlstudy-phys/data/{filename} ./
with gzip.open(filename, 'rb') as f:
  app3000_grant6000_W = pickle.load(f)

filename = "app3000_grant6000_B.pkl.gz"
!gsutil cp gs://mlstudy-phys/data/{filename} ./
with gzip.open(filename, 'rb') as f:
  app3000_grant6000_B = pickle.load(f)

class Model(object):
    def __init__(self, W, B):
        self.W = W
        self.B = B 
        self.variables = [ self.W, self.B ]
    
    def frwrd_pass(self,X_train):
        out = tf.matmul( X_train, self.W ) + self.B
        normalized_out = tf.nn.l2_normalize(out, axis=1)
        return normalized_out

model = Model(W=app3000_grant6000_W, B=app3000_grant6000_B)

filename = "grants_tf_feature_dict_6000.pkl.gz"
!gsutil cp gs://mlstudy-phys/data/{filename} ./
with gzip.open(filename, 'rb') as f:
  grants_tf_feature_dict_6000 = pickle.load(f)

filename = "grants_6000_kind_title.pkl.gz"
!gsutil cp gs://mlstudy-phys/data/{filename} ./
with gzip.open(filename, 'rb') as f:
  grants_6000_kind_title = pickle.load(f)

kind_dict = { 
    k: v for k,v in zip(grants_6000_kind_title['parsed'], grants_6000_kind_title['kind_code'])
}

title_dict = { 
    k: v for k,v in zip(grants_6000_kind_title['parsed'], grants_6000_kind_title['title'])
}

Copying gs://mlstudy-phys/data/app3000_grant6000.model...
/ [1 files][167.2 MiB/167.2 MiB]                                                
Operation completed over 1 objects/167.2 MiB.                                    


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Copying gs://mlstudy-phys/data/app3000_grant6000_W.pkl.gz...
/ [1 files][ 75.8 KiB/ 75.8 KiB]                                                
Operation completed over 1 objects/75.8 KiB.                                     
Copying gs://mlstudy-phys/data/app3000_grant6000_B.pkl.gz...
/ [1 files][  815.0 B/  815.0 B]                                                
Operation completed over 1 objects/815.0 B.                                      
Copying gs://mlstudy-phys/data/grants_tf_feature_dict_6000.pkl.gz...
/ [1 files][  2.5 MiB/  2.5 MiB]                                                
Operation completed over 1 objects/2.5 MiB.                                      
Copying gs://mlstudy-phys/data/grants_6000_kind_title.pkl.gz...
/ [1 files][180.6 KiB/180.6 KiB]                                                
Operation completed over 1 objects/180.6 KiB.                                    


#3. Input search word
Claim text is a necessary input. You can input single claim, multiple claims and/or arbitrary text describing the features of the invention.

Background of the invention and IPC (International Patent Classification) are optional.
Background of the invention can be an arbitrary text describing related technology of the invention.
In this model, you can choose major IPC codes upto three categories. For your information, brief desctiptions of IPC code are listed below:

|Code | Brief description|
|---|---|
|A | HUMAN NECESSITIES|
|B | PERFORMING OPERATIONS; TRANSPORTING|
|C| CHEMISTRY; METALLURGY|
|D| TEXTILES; PAPER|
|E| FIXED CONSTRUCTIONS|
|F| MECHANICAL ENGINEERING; LIGHTING; HEATING; WEAPONS; BLASTING|
|G| PHYSICS|
|H| ELECTRICITY|

In [0]:
#@title Input calim, background of the invention(optional), and IPC(optional) { display-mode: "form" }

claim = 'An electronic percussion instrument comprising: a frame; a head, which is made of an elastic material with a higher flexibility than the frame, wherein the head has a main area including a main strike area, which is disposed in a front side of the frame and which is mainly subjected to a striking operation, and an auxiliary area which includes a fixing part is fixed to the frame but does not include the main strike area; and an impact sensor which converts a vibration occurring on the main strike area subjected to a striking operation into an electric signal, wherein at least part of the main area of the head is connected to the auxiliary area via a connecting part which is reduced in thickness in comparison with the auxiliary area.' #@param {type:"string"}
background_of_the_invention = "[0001] 1. Field of the Invention [0002] The present invention relates to an electronic percussion instrument including an impact sensor which converts vibration of a head being struck with a beater into an electric signal so as to generate an electronic musical sound. [0003] The present application claims priority on Japanese Patent Application No. 2013-49035 and Japanese Patent Application No. 2014-47225, the entire content of which is incorporated herein by reference. [0004] 2. Description of the Related Art [0005] Conventionally-known electronic percussion instruments are designed to generate an electronic musical sound based on an electric signal output from an impact sensor which detects vibration of a head being struck with a beater. Patent Literature Document 1 (PLT1) discloses an electronic percussion instrument serving as an electronic bass drum with a circular head, made of an elastic material, whose periphery is engaged with a frame. An impact sensor is attached to the back of a strike area corresponding to the center of a head via a center cushion with an outer periphery encompassed by a ring-shaped damper cushion. A vibrating wave occurs when the strike area of a head is being struck with a beater. A vibrating wave is transmitted toward the periphery of a head, bounced back, and then attenuated by the damper cushion. [0006] The electronic percussion instrument of PLT1 generates an impulsive sound (i.e. a sound directly caused by an impact of a head being struck with a beater) independently of an electronic musical sound which is generated based on an electric signal output from an impact sensor which detects vibration occurring on a head being struck with a beater. Due to an impact of a head, a large vibration occurs in the entirety of the internal area of a head (i.e. an area which exists inwardly of the periphery of a head) compared to the periphery of a head which is fixed to the frame, thus causing a large impulsive sound. A large impulsive sound accompanied with an electronic musical sound is offensive to human's ears, and therefore an impulsive sound may degrade the sound quality of an electronic percussion instrument in terms of articulation. [0007] It is necessary to reduce an impulsive sound accompanied with an electronic musical sound because both the pitch and the tone color of an impulsive sound significantly affect the sound quality of an electronic percussion instrument. For example, it is not preferable for an electronic percussion instrument to generate a high-pitch impulsive sound with a low volume which may degrade sound quality." #@param {type:"string"}

if background_of_the_invention == "":
  background_of_the_invention = "Backgraound of the invention."

IPC_1 = 'G' #@param ["", "A", "B", "C", "D", "E", "F", "G", "H"]
IPC_2 = '' #@param ["", "A", "B", "C", "D", "E", "F", "G", "H"]
IPC_3 = '' #@param ["", "A", "B", "C", "D", "E", "F", "G", "H"]


#4. Calculate the search result

You can set the number of documents.

Distance represents how close to the inputted invention.
The results are sorted by distance.

In [0]:
#@title Show the result { display-mode: "form" }

def text_to_vec(text):
    words = text.split()
    filtered_words = []
    
    for word in words:
        if word in w2v.wv.vocab:
            filtered_words.append(word)
    vec = np.average(w2v[filtered_words], axis=0)
    return vec

tmp_vec = np.concatenate([text_to_vec(claim) , text_to_vec(background_of_the_invention)])

section_to_vec_dict = {
    "A":[0,0,0,0,0,0,0,1],
    "B":[0,0,0,0,0,0,1,0],
    "C":[0,0,0,0,0,1,0,0],
    "D":[0,0,0,0,1,0,0,0],
    "E":[0,0,0,1,0,0,0,0],
    "F":[0,0,1,0,0,0,0,0],
    "G":[0,1,0,0,0,0,0,0],
    "H":[1,0,0,0,0,0,0,0],    
}

def section_to_vec(list):
  vec = np.zeros(8)
  for section in list:
    if section not in section_to_vec_dict.keys():
      vec = vec + np.zeros(8)
    else:
      vec = vec + np.array(section_to_vec_dict[section])
  return vec/np.linalg.norm(vec) if np.linalg.norm(vec) >0 else vec

IPC = list(IPC_1)+list(IPC_2)+list(IPC_3)
IPC_vec = section_to_vec(IPC)

feature_vec = np.concatenate([tmp_vec, IPC_vec])
feature_vec = feature_vec/np.linalg.norm(feature_vec)
feature_vec = feature_vec.reshape([1, len(feature_vec)])

input_tensor = tf.convert_to_tensor(feature_vec, dtype=tf.float32)
tf_vec = model.frwrd_pass(input_tensor)

def angle_distance(a, b):
    with tf.name_scope('dist'):
        prods = tf.math.multiply(a, b)
        angle = tf.math.acos(tf.reduce_sum(prods))
        out = tf.math.divide(angle, tf.constant(math.pi))

        return out + 1e-12

results_dict = {
    k:angle_distance(tf_vec[0], v).numpy() for k,v 
    in grants_tf_feature_dict_6000.items()
}

sorted_results = sorted(results_dict.items(), key=lambda x:x[1])

Top_K =  50#@param {type:"number"}

grid = widgets.Grid(Top_K+1, 4, header_row=True, header_column=True)
with grid.output_to(0, 0):
  print("Ranking")

with grid.output_to(0, 1):
  print("Title of the invention")

with grid.output_to(0, 2):
  print("Google Patents URL")

with grid.output_to(0, 3):
  print("distance")

for i, (num, dist) in enumerate(sorted_results[0:Top_K]):
  with grid.output_to(i+1, 0):
    print(i+1)

  with grid.output_to(i+1, 1):
    title = title_dict[num]
    print(title+"\n")

  with grid.output_to(i+1, 2):
    kind = kind_dict[num]
    s = urllib.parse.quote('patents.google.com/patent/US'+str(num)+str(kind))
    print('https://'+s)

  with grid.output_to(i+1, 3):
    print(dist)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,
,,,


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.36813936


<IPython.core.display.Javascript object>